In [1]:
def basis_reconciliation(alice, bob):
    """
        alice: {no. : [bit encoded, basis chosen to encode in ]}
        bob  : {no. : [bit after measurement, basis chosen to measure in]}
        
        First check if the length of both lists are the same
            -> if yes, keep only those bits for alice and bob for which
               the basis encoded in and measured in is the same. 
    """
    basis_bit_alice = list(alice.values())
    basis_bit_bob = list(bob.values())
    
    if len(basis_bit_alice) == len(basis_bit_bob):
        raw_key_alice = []
        raw_key_bob = []

        for i in range(len(basis_bit_alice)):
            if basis_bit_alice[i][1] == basis_bit_bob[i][1]:
                raw_key_alice.append(basis_bit_alice[i][0])
                raw_key_bob.append(basis_bit_bob[i][0])

        return raw_key_alice, raw_key_bob
    
    else:
        return None, None

In [2]:
alice = {1:[0,0],2:[0,1],3:[0,0]}
bob = {1:[1,0],2:[0,0],3:[1,0]}

In [3]:
basis_reconciliation(alice, bob)

([0, 0], [1, 1])

In [4]:
class Alice:
    def __init__(self, n):
        self.n = n
        self.alice = {} #{no. : [bit encoded, basis chosen to encode in ]} no. is some unique number
                        #{1:[0,0],2:[0,1],3:[0,0]} Example

    def generate_and_encode(self): #TODO Dighvijay
        """
            Will generate n bits randomly
            For each bit generated, a basis is chosen in which it is encoded
            Dependency for encoding: <class LinearPolarizer>
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.alice mentioned above
        """
        pass
    

class Bob:
    def __init__(self, n):
        self.n = n
        self.bob = {} #{no. : [bit after measurement, basis chosen to measure in]}
                      #{1:[1,0],2:[0,0],3:[1,0]} Example
    
    def choose_basis_and_measure(self, received): #TODO Dighvijay
        """
            received : the data received by bob
            Dependency for measurement: <class PolarizingBeamSplitter>
            
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.bob mentioned above
        """
        pass
    
    